In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

from albk.data.utils import idx_to_locate
use_disjoint_files = False
from torch.utils.data import TensorDataset, DataLoader


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product

In [3]:
def get_common_label_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    common_files = set(f1_base_files).intersection(f2_base_files)
    common_label_files = []
    for file in common_files:
        ds1 = xr.open_dataset(join(path1, file))
        ds2 = xr.open_dataset(join(path2, file))
        if np.all(ds1.label.values == ds2.label.values):
            common_label_files.append(file)
    
    return list(map(lambda f: join(path1, f), common_label_files))

def get_disjoint_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    disjoint_files = set(f1_base_files).symmetric_difference(f2_base_files)
    
    f1_disjoint = [f for f in disjoint_files if f in f1_base_files]
    f1_disjoint = list(map(lambda f: join(path1, f), f1_disjoint))

    f2_disjoint = [f for f in disjoint_files if f in f2_base_files]
    f2_disjoint = list(map(lambda f: join(path2, f), f2_disjoint))
    
    return f1_disjoint + f2_disjoint

In [4]:
base_path = expanduser("~/bangladesh_labels/bkdb/bangladesh_labels")
paths = {"zeel": ("vannsh", "rishabh"), "rishabh": ("suraj", "dhruv"), "suraj": ("aditi", "madhav")}

all_labeled_files = []
for moderator, annotators in paths.items():
    # Get moderator files
    moderator_path = join(base_path, "moderated", moderator)
    moderator_files = glob(join(moderator_path, "*.nc"))
    
    # Get annotator common label files
    annotator1_path = join(base_path, annotators[0])
    annotator2_path = join(base_path, annotators[1])
    
    common_base_files = get_common_label_files(annotator1_path, annotator2_path)
    
    # Get disjoint files
    disjoint_files = get_disjoint_files(annotator1_path, annotator2_path)
    
    all_files = moderator_files + common_base_files
    if use_disjoint_files:
        all_files.extend(disjoint_files)
    assert len(all_files) == len(set(all_files))
    all_labeled_files.extend(all_files)
    
    print("Moderator", moderator)
    print(" "*5, "Moderator files", len(moderator_files))
    print(" "*5, "Common label files", len(common_base_files))
    print(" "*5, "Disjoint files", len(disjoint_files))
    print(" "*5, f"Total files from {moderator} and {annotators}", len(all_files))
    print(" "*5, "Total annotatated files", len(all_labeled_files))
    
print("Total dataset size", len(all_labeled_files) * 25)

Moderator zeel
      Moderator files 88
      Common label files 359
      Disjoint files 662
      Total files from zeel and ('vannsh', 'rishabh') 447
      Total annotatated files 447
Moderator rishabh
      Moderator files 98
      Common label files 115
      Disjoint files 736
      Total files from rishabh and ('suraj', 'dhruv') 213
      Total annotatated files 660
Moderator suraj
      Moderator files 195
      Common label files 165
      Disjoint files 746
      Total files from suraj and ('aditi', 'madhav') 360
      Total annotatated files 1020
Total dataset size 25500


In [5]:
print(all_labeled_files[:5])


['/home/rishabh.mondal/bangladesh_labels/bkdb/bangladesh_labels/moderated/zeel/24.90,90.77.nc', '/home/rishabh.mondal/bangladesh_labels/bkdb/bangladesh_labels/moderated/zeel/24.58,91.69.nc', '/home/rishabh.mondal/bangladesh_labels/bkdb/bangladesh_labels/moderated/zeel/24.44,90.83.nc', '/home/rishabh.mondal/bangladesh_labels/bkdb/bangladesh_labels/moderated/zeel/24.63,88.25.nc', '/home/rishabh.mondal/bangladesh_labels/bkdb/bangladesh_labels/moderated/zeel/24.99,89.79.nc']


In [6]:
def get_bk_stats(path):
    ds = xr.open_dataset(path)
    z = (ds.label.values == "Z").sum()
    f = (ds.label.values == "F").sum()
    o = (ds.label.values == "O").sum()
    return {"Z": z, "F": f, "O": o}

df = pd.DataFrame([get_bk_stats(path) for path in all_labeled_files])

df_sum = df.sum(axis=0)
print("Total ZicZac Brick Kilns", df_sum["Z"])
print("Total Fixed Chimney Brick Kilns", df_sum["F"])
print("All Brick Kilns", df_sum["Z"] + df_sum["F"])
print("All Non-brick Kilns", df_sum["O"])

Total ZicZac Brick Kilns 1285
Total Fixed Chimney Brick Kilns 412
All Brick Kilns 1697
All Non-brick Kilns 23803


In [7]:
# save_path = expanduser("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/bangladesh_labels/")
# os.system(f"rm -rf {save_path}")
# os.makedirs(save_path)

# def copy_file(path):
#     copy(path, save_path)
    
# _ = Parallel(n_jobs=20)(delayed(copy_file)(path) for path in tqdm(all_labeled_files))



In [8]:
images_path = expanduser("/home/patel_zeel/bkdb/bangladesh/")
# load_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/temporary"
files = all_labeled_files
# print(files)
print(len(files))



1020


In [9]:
def get_index_and_image(file):
    index = []
    images = []
    labels = []
    base_name = basename(file)
    # print(base_name)
    image_path = join(images_path, base_name).replace(".nc", ".zarr")
    print(image_path)
    # print()
    label_ds = xr.open_dataset(file)
    # print (label_ds)
    image_ds = xr.open_zarr(image_path, consolidated=False)
    # image = image_ds.data.reshape(-1, 224, 224, 3)
    for lat_lag, lon_lag in product(range(-2, 3), repeat=2):
        index.append(base_name.replace(".nc", "")+f"_{lat_lag}_{lon_lag}")
        images.append(torch.tensor(image_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['data'].values)[np.newaxis, ...])
        labels.append(torch.tensor((label_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['label'].values != "O").astype(np.uint8)))
        
    return index, images, labels



def get_data():
    out = Parallel(n_jobs=32)(delayed(get_index_and_image)(file) for file in tqdm(files, total=len(files)))
    index = np.concatenate([np.array(idx) for idx, _, _ in out])
    images = torch.concat([torch.einsum("nhwc->nchw", torch.concat(imgs)) for _, imgs, _ in out])
    # scale
    # images = images / 255
    # mean normalize
    # images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
    
    labels = np.concatenate([np.array(lbl) for _, _, lbl in out])
    labels = torch.tensor(labels, dtype=torch.uint8)
    #check the all dytpes
    print(index.dtype, images.dtype, labels.dtype)
    return index, images, labels

index, images, labels = get_data()
print(images.dtype)
print(index.shape, images.shape, labels.shape)    

  3%|▎         | 32/1020 [00:00<00:15, 65.25it/s]

/home/patel_zeel/bkdb/bangladesh/24.44,90.83.zarr
/home/patel_zeel/bkdb/bangladesh/25.55,89.29.zarr
/home/patel_zeel/bkdb/bangladesh/24.63,88.25.zarr
/home/patel_zeel/bkdb/bangladesh/24.99,89.79.zarr
/home/patel_zeel/bkdb/bangladesh/24.46,89.98.zarr
/home/patel_zeel/bkdb/bangladesh/24.91,90.81.zarr
/home/patel_zeel/bkdb/bangladesh/24.58,91.69.zarr
/home/patel_zeel/bkdb/bangladesh/25.12,89.41.zarr
/home/patel_zeel/bkdb/bangladesh/24.72,89.58.zarr
/home/patel_zeel/bkdb/bangladesh/24.91,89.38.zarr
/home/patel_zeel/bkdb/bangladesh/24.54,88.84.zarr
/home/patel_zeel/bkdb/bangladesh/24.45,90.82.zarr
/home/patel_zeel/bkdb/bangladesh/25.78,89.31.zarr
/home/patel_zeel/bkdb/bangladesh/24.90,90.77.zarr
/home/patel_zeel/bkdb/bangladesh/24.60,90.43.zarr
/home/patel_zeel/bkdb/bangladesh/24.58,90.01.zarr
/home/patel_zeel/bkdb/bangladesh/24.66,88.26.zarr
/home/patel_zeel/bkdb/bangladesh/24.77,89.90.zarr
/home/patel_zeel/bkdb/bangladesh/24.93,90.37.zarr
/home/patel_zeel/bkdb/bangladesh/24.85,91.73.zarr


  6%|▋         | 64/1020 [00:02<00:36, 25.88it/s]

/home/patel_zeel/bkdb/bangladesh/24.63,90.29.zarr
/home/patel_zeel/bkdb/bangladesh/24.56,89.56.zarr
/home/patel_zeel/bkdb/bangladesh/24.92,89.97.zarr
/home/patel_zeel/bkdb/bangladesh/25.79,89.04.zarr
/home/patel_zeel/bkdb/bangladesh/24.78,89.40.zarr
/home/patel_zeel/bkdb/bangladesh/25.48,89.53.zarr
/home/patel_zeel/bkdb/bangladesh/24.52,88.23.zarr
/home/patel_zeel/bkdb/bangladesh/25.98,89.44.zarr
/home/patel_zeel/bkdb/bangladesh/24.92,89.79.zarr
/home/patel_zeel/bkdb/bangladesh/24.98,89.84.zarr
/home/patel_zeel/bkdb/bangladesh/25.65,89.40.zarr
/home/patel_zeel/bkdb/bangladesh/24.76,88.68.zarr
/home/patel_zeel/bkdb/bangladesh/24.44,90.67.zarr
/home/patel_zeel/bkdb/bangladesh/25.63,89.63.zarr
/home/patel_zeel/bkdb/bangladesh/25.05,89.37.zarr
/home/patel_zeel/bkdb/bangladesh/25.77,88.84.zarr
/home/patel_zeel/bkdb/bangladesh/26.33,89.01.zarr
/home/patel_zeel/bkdb/bangladesh/25.07,90.88.zarr
/home/patel_zeel/bkdb/bangladesh/25.55,88.99.zarr
/home/patel_zeel/bkdb/bangladesh/25.41,89.41.zarr


  9%|▉         | 96/1020 [00:02<00:26, 35.52it/s]

...

 94%|█████████▍| 960/1020 [00:15<00:00, 67.82it/s]

/home/patel_zeel/bkdb/bangladesh/24.27,89.41.zarr
/home/patel_zeel/bkdb/bangladesh/24.22,91.50.zarr
/home/patel_zeel/bkdb/bangladesh/23.67,90.43.zarr
/home/patel_zeel/bkdb/bangladesh/24.22,90.93.zarr
/home/patel_zeel/bkdb/bangladesh/24.30,90.56.zarr
/home/patel_zeel/bkdb/bangladesh/24.41,89.65.zarr
/home/patel_zeel/bkdb/bangladesh/24.24,89.58.zarr
/home/patel_zeel/bkdb/bangladesh/24.03,90.30.zarr
/home/patel_zeel/bkdb/bangladesh/24.40,88.51.zarr
/home/patel_zeel/bkdb/bangladesh/23.49,90.62.zarr
/home/patel_zeel/bkdb/bangladesh/24.05,91.22.zarr
/home/patel_zeel/bkdb/bangladesh/23.60,89.18.zarr
/home/patel_zeel/bkdb/bangladesh/24.11,89.48.zarr
/home/patel_zeel/bkdb/bangladesh/23.97,90.05.zarr
/home/patel_zeel/bkdb/bangladesh/23.90,88.90.zarr
/home/patel_zeel/bkdb/bangladesh/24.41,89.45.zarr
/home/patel_zeel/bkdb/bangladesh/24.08,90.94.zarr
/home/patel_zeel/bkdb/bangladesh/23.89,90.05.zarr
/home/patel_zeel/bkdb/bangladesh/23.69,89.70.zarr
/home/patel_zeel/bkdb/bangladesh/23.82,91.18.zarr


100%|██████████| 1020/1020 [00:15<00:00, 64.64it/s]

/home/patel_zeel/bkdb/bangladesh/23.55,89.21.zarr
/home/patel_zeel/bkdb/bangladesh/24.43,91.92.zarr
/home/patel_zeel/bkdb/bangladesh/23.64,90.43.zarr
/home/patel_zeel/bkdb/bangladesh/23.71,90.15.zarr
/home/patel_zeel/bkdb/bangladesh/23.57,89.13.zarr
/home/patel_zeel/bkdb/bangladesh/23.56,89.79.zarr
/home/patel_zeel/bkdb/bangladesh/23.67,90.22.zarr
/home/patel_zeel/bkdb/bangladesh/24.08,88.89.zarr
/home/patel_zeel/bkdb/bangladesh/24.11,90.59.zarr
/home/patel_zeel/bkdb/bangladesh/24.09,90.85.zarr
/home/patel_zeel/bkdb/bangladesh/23.92,90.64.zarr
/home/patel_zeel/bkdb/bangladesh/23.96,89.84.zarr
/home/patel_zeel/bkdb/bangladesh/23.84,89.83.zarr
/home/patel_zeel/bkdb/bangladesh/24.11,90.10.zarr
/home/patel_zeel/bkdb/bangladesh/23.83,89.01.zarr


/home/patel_zeel/bkdb/bangladesh/23.83,90.02.zarr
/home/patel_zeel/bkdb/bangladesh/23.99,89.21.zarr
/home/patel_zeel/bkdb/bangladesh/23.75,90.54.zarr
/home/patel_zeel/bkdb/bangladesh/23.76,90.20.zarr
/home/patel_zeel/bkdb/bangladesh/23.99,89.23.zarr
/home/patel_zeel/bkdb/bangladesh/23.96,89.25.zarr
/home/patel_zeel/bkdb/bangladesh/24.06,91.00.zarr
/home/patel_zeel/bkdb/bangladesh/23.50,90.97.zarr
/home/patel_zeel/bkdb/bangladesh/23.55,89.20.zarr
/home/patel_zeel/bkdb/bangladesh/23.50,90.95.zarr
/home/patel_zeel/bkdb/bangladesh/24.07,90.63.zarr
/home/patel_zeel/bkdb/bangladesh/23.90,89.62.zarr
/home/patel_zeel/bkdb/bangladesh/24.28,89.00.zarr
/home/patel_zeel/bkdb/bangladesh/24.04,90.33.zarr
/home/patel_zeel/bkdb/bangladesh/24.00,91.12.zarr
/home/patel_zeel/bkdb/bangladesh/23.93,90.15.zarr
/home/patel_zeel/bkdb/bangladesh/23.47,91.14.zarr
/home/patel_zeel/bkdb/bangladesh/24.18,91.50.zarr
/home/patel_zeel/bkdb/bangladesh/24.17,89.43.zarr
/home/patel_zeel/bkdb/bangladesh/23.60,89.11.zarr


In [11]:
print(images.dtype)
print(labels.dtype)

torch.uint8
torch.uint8


### dont run the below cell otherwise it will overwrite the data

In [12]:
# save the tensors data 
# print(images.dtype)
# save_path="/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt"
# torch.save({
#     'index': index,
#     'images': images,
#     'labels': labels
# }, save_path)

### You can run the below cell if you want to access the data

In [3]:
# Load the saved tensors
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")

# Access the tensors
index = loaded_data['index']
images = loaded_data['images']
labels = loaded_data['labels']


In [4]:
print(index.shape, images.shape, labels.shape)  

(10025,) torch.Size([10025, 3, 224, 224]) torch.Size([10025])


In [5]:
images.dtype, labels.dtype, index.dtype

(torch.uint8, torch.uint8, dtype('<U17'))

### For mannual splitting of data into train and test set



### Stratified Cross Validation spliting of data into train and test set

In [6]:
import torch
from collections import Counter

fold_data = []  # List to store data from each fold

seed = 42  # Use your desired random seed
splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
images = images / 255
    # mean normalize
images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
for fold, (train_idx, test_idx) in enumerate(splitter.split(images, labels)):
    X_train, X_test = images[train_idx], images[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    # Count occurrences of each class in train and test sets
    train_counter = Counter(y_train.numpy())
    test_counter = Counter(y_test.numpy())
    print(train_counter)
    print(test_counter)
    print(f"Fold {fold + 1} - Train: {train_counter}, Test: {test_counter}")

    fold_data.append({
        'fold': fold + 1,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'train_counter': train_counter,
        'test_counter': test_counter
    })

# Save the list of fold data using torch.save
# torch.save(fold_data, 'f/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/cross_val_data/fold_data.pt')


Counter({0: 6737, 1: 781})
Counter({0: 2246, 1: 261})
Fold 1 - Train: Counter({0: 6737, 1: 781}), Test: Counter({0: 2246, 1: 261})
Counter({0: 6737, 1: 782})
Counter({0: 2246, 1: 260})
Fold 2 - Train: Counter({0: 6737, 1: 782}), Test: Counter({0: 2246, 1: 260})
Counter({0: 6737, 1: 782})
Counter({0: 2246, 1: 260})
Fold 3 - Train: Counter({0: 6737, 1: 782}), Test: Counter({0: 2246, 1: 260})
Counter({0: 6738, 1: 781})
Counter({0: 2245, 1: 261})
Fold 4 - Train: Counter({0: 6738, 1: 781}), Test: Counter({0: 2245, 1: 261})
